In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400
import graphviz
import copy

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format #only concern 2 digits after the decimal point
pd.set_option('display.width', 75) #safer option than .set_option.display.width bc regex will match to multiple option names
pd.set_option('display.max_columns', 10)

In [ ]:
# path is /content/drive/MyDrive/Coding_projects/Chang-COVID

#(IGNORE) Read in ZC SNI CSV

In [ ]:
ZC_SNI_premerge = pd.read_csv('/content/drive/MyDrive/Coding_projects/Chang-COVID/zip_code_sni.csv',\
                              names=['zip_code','sni_2020_index','sni_rank'],\
                              skiprows=1,\
                              low_memory=False)

In [ ]:
type(ZC_SNI_premerge)

In [ ]:
ZC_SNI_premerge.head(10)

In [ ]:
ZC_SNI_premerge.dtypes

In [ ]:
ZC_SNI_premerge.shape

In [ ]:
ZC_SNI_premerge.index

In [ ]:
ZC_SNI_premerge.index.nunique()

In [ ]:
ZC_SNI_premerge.info()

In [ ]:
ZC_SNI_premerge = ZC_SNI_premerge.astype({'zip_code':'category','sni_rank':'category'})
ZC_SNI_premerge.info()

In [ ]:
ZC_SNI_predict = ZC_SNI_premerge
type(ZC_SNI_predict)

In [ ]:
ZC_SNI_predict.head(10)

#(IGNORE) Make Copies of OG `ptinfo_ts0 -> ts1` and `ZC_SNI_predict -> predict1` (cleaned/recategorized imports) (ignore)

In [ ]:
import copy

In [ ]:
ptinfo_ts1 = ptinfo_ts0.copy(deep=True)
ZC_SNI_predict1 = ZC_SNI_predict.copy(deep=True)


In [ ]:
ZC_SNI_predict1.head(10)

In [ ]:
ptinfo_ts0.head(10)

# (IGNORE) Convert ZC_SNI to a true dictionary `ZC_SNI_dictionary` and Create new SNI DF Column using it


In [ ]:
#Personal: Convert series (Zip Code column) to dictionary
#link: https://cmdlinetips.com/2021/04/convert-two-column-values-from-pandas-dataframe-to-a-dictionary/#:~:text=Another%20approach%20to%20convert%20two,all%20columns%20in%20the%20dataframe.
ZC_SNI_dictionary = ZC_SNI_predict1.set_index('zip_code').to_dict()['sni_rank']

In [ ]:
type(ZC_SNI_dictionary)

In [ ]:
print(ZC_SNI_dictionary)

In [ ]:
#pt_SNI_merged = pd.merge(ptinfo_premerge_copy1,ZC_SNI_premerge_copy1,how='left',on='zip_code')

In [ ]:
#pt_SNI_merged.shape

In [ ]:
#df['Capital'] = df['Country'].map(country_capital)
#Personal: Create new column via map() 
ptinfo_ts1.info()



In [ ]:
ptinfo_ts1

# 1. Read in PT Info CSV File and Clean for Analysis

In [ ]:
#this file does not have leap year values
#REMINDER: CHECK IF THE PARSE DATES FUNCTION WORKED OR NOT
ptinfo_premerge = pd.read_csv('/content/drive/MyDrive/Coding_projects/Chang-COVID/ptinfo_premerge.csv',\
                              names=['date_time','visit_id','mrn','age','gender','race','ethnicity','zip_code'],\
                              index_col='date_time',\
                              skiprows=1,\
                              parse_dates=['date_time'],
                              low_memory=False)


In [ ]:
type(ptinfo_premerge)

## 1A. Small Exploratory Data Analysis on Newly Read File (numerical exploration bc all #s rn)

In [ ]:
ptinfo_premerge.head(5)

In [ ]:
ptinfo_premerge.head(5).T #first 5 rows but now oriented vertically

In [ ]:
ptinfo_premerge.shape

In [ ]:
ptinfo_premerge.index

In [ ]:
ptinfo_premerge.index.nunique()

In [ ]:
ptinfo_premerge.dtypes

In [ ]:
ptinfo_premerge.describe() #for all columns with # values

In [ ]:
ptinfo_premerge.sample(2, random_state=1).T

In [ ]:
ptinfo_premerge = ptinfo_premerge.sort_index()

In [ ]:
ptinfo_premerge.corr()

In [ ]:
#correlation map
#esp good bc rn all categoricals are encoded by NUMERICAL variables
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(ptinfo_premerge.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

# 2. Defined all dummy values from hospital glossary for [Gender, Race, Ethnicity] via dictionary mapping
## Note: Renamed working Dataframe from `ptinfo_premerge` %>% `pt_info_partmap` 

In [ ]:
#define dictionaries
r_map = {'gender':{0:'Male', 1:'Female', 2:'Unknown'},\
         'race':{0:'White',1:'Black',2:'Asian',3:'American_Indian',4:'Eastern_Indian',\
                 5:'Hispanic',6:'Other',7:'Other_Pacific_Islander',8:'Unknown'},\
         'ethnicity':{0:'Non_Hispanic',1:'Hispanic',2:'Unknown'}}
print(r_map)
type(r_map)


In [ ]:
#r_map = {'state':{'1':'tn', '2':'kerala'},'country':{'67':'usa', '52':'india'}}
#df.replace(r_map)
#.replace() is more forgiving than .map() bc it skips over unused dict keys

pt_info_partmap = ptinfo_premerge.replace(r_map)
pt_info_partmap.head(10)

In [ ]:
zip_code_dict = {'75041':'5','75042':'5','75051':'5','75060':'5','75061':'5','75141':'5','75172':'5','75180':'5','75203':'5','75210':'5','75211':'5','75212':'5','75215':'5','75216':'5','75217':'5','75220':'5','75223':'5','75224':'5','75227':'5','75228':'5','75231':'5','75232':'5','75233':'5','75236':'5','75237':'5','75240':'5','75241':'5','75246':'5','75247':'5','75253':'5','75006':'4','75040':'4','75050':'4','75062':'4','75116':'4','75134':'4','75146':'4','75149':'4','75150':'4','75159':'4','75208':'4','75226':'4','75235':'4','75243':'4','75038':'3','75043':'3','75044':'3','75052':'3','75115':'3','75137':'3','75234':'3','75238':'3','75249':'3','75251':'3','75254':'3','75080':'2','75081':'2','75104':'2','75181':'2','75204':'2','75206':'2','75207':'2','75229':'2','75001':'1','75019':'1','75039':'1','75048':'1','75063':'1','75088':'1','75089':'1','75182':'1','75201':'1','75202':'1','75205':'1','75209':'1','75214':'1','75218':'1','75219':'1','75225':'1','75230':'1','75244':'1','75248':'1'}
print(zip_code_dict)


In [ ]:
type(zip_code_dict)

In [ ]:
pt_info_partmap.head(5)

In [ ]:
#Personal: Map values in column 'zip code' to values in 'zip code dict' -> assign to new column name
# link to idea: https://kanoki.org/2019/04/06/pandas-map-dictionary-values-with-dataframe-columns/ 
pt_info_partmap['sni_rank'] = pt_info_partmap['zip_code'].map(zip_code_dict)

In [ ]:
pt_info_partmap.dtypes

In [ ]:
#Quality check to make sure object 
objcols = pt_info_partmap.select_dtypes(include=["object"]).columns
pt_info_partmap[objcols].isnull()

## 2A. Small Exploratory Data Analysis on Categorical Variables (that we just mapped)
Note: Lots of `value_counts` on each categ vars

In [ ]:
pt_info_partmap.head(2)

In [ ]:
pt_info_partmap.info()

In [ ]:
pt_info_partmap.gender.value_counts(normalize=True)

In [ ]:
pt_info_partmap.race.value_counts(normalize=True)

In [ ]:
pt_info_partmap.ethnicity.value_counts(normalize=True)

In [ ]:
pt_info_partmap.zip_code.value_counts(normalize=True)

In [ ]:
pt_info_partmap.sni_rank.value_counts(normalize=True)

In [ ]:
pt_info_partmap.describe()

# 3. Filtered for Dallas County Zip Codes with Dallas-SNI Dictionary by removing/dropping all rows with NaT values

In [ ]:
# Use isnull() to select all rows with NaN in an entire DataFrame
# df[df.isnull().any(axis=1)]
pt_info_partmap[pt_info_partmap.isnull().any(axis=1)]

In [ ]:
pt_info_partmap.shape

In [ ]:
#Personal: Drop rows that were not mapped to dictionary (not Dallas county)
pt_info_partmap = pt_info_partmap.dropna()
pt_info_partmap.shape

In [ ]:
pt_info_partmap.head(3)

In [ ]:
#DO NOT RUN IF INDEX IS `DATE_TIME`
#pt_info_partmap = pt_info_partmap.reset_index(drop=True) 


In [ ]:
#Sort Indexing is IMPORTANT for later time series - ARIMA wont work if its not sorted
pt_info_partmap.sort_index() 

In [ ]:
pt_info_partmap.index.nunique() #arrival date_times are NOT unique - 10664 replicates

In [ ]:
duplicate_row_values = 93003 - 82340
print(duplicate_row_values) #10,663 duplicate row values

In [ ]:
pt_info_partmap.info()

In [ ]:
pt_info_partmap = pt_info_partmap.astype({'visit_id':'float64',\
                                          'mrn':'float64',\
                                          'age':'category',\
                                          'gender':'category',\
                                          'race':'category',\
                                          'ethnicity':'category',\
                                          'zip_code':'category',\
                                          'sni_rank':'category'})
pt_info_partmap.info()

## 3A. EDA for Cleaned and Labeled Dataframe

In [ ]:
# show the names of columns with category data type and check for number of missings
catcols = pt_info_partmap.select_dtypes(include=["category"]).columns
pt_info_partmap[catcols].isnull().sum() #0 is false, so no null sums

In [ ]:
#describe for continuous quantitative vars
#value counts for each categorical vars (can be numerical or str)
pt_info_partmap.age.value_counts(normalize=True)


In [ ]:
pt_info_partmap.gender.value_counts(normalize=True)


In [ ]:
pt_info_partmap.race.value_counts(normalize=True)


In [ ]:
pt_info_partmap.ethnicity.value_counts(normalize=True)


In [ ]:
pt_info_partmap.zip_code.value_counts(normalize=True)


In [ ]:
pt_info_partmap.sni_rank.value_counts(normalize=True)


In [ ]:
# summary stats for continuous variables
pt_info_partmap.describe()

In [ ]:
pt_info_partmap.corr()

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(pt_info_partmap.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

# 4. Saved Checkpoint via *deep copy* of `pt_info_partmap` -> begin time series with `ptinfo_ts0`

In [ ]:
#Personal: Deep Copied dataframe from pt_info_partmap -> ptinfo_ts0
ptinfo_ts0 = pt_info_partmap.copy(deep=True)